# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

from sqlalchemy import create_engine

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
from sklearn.model_selection import train_test_split
import nltk
nltk.download(['punkt', 'wordnet'])



[nltk_data] Downloading package punkt to /Users/zohir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zohir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 1. Load messages dataset from the database

In [2]:
# Look for the table name

db_file_name = 'DisasterResponse.db'
engine = create_engine('sqlite:///'+db_file_name)
print (engine.table_names())


['disasters_messages']


In [3]:
# load data from database
db_file_name = 'DisasterResponse.db'
db_table_name = 'disasters_messages'

df = pd.read_sql_table(db_table_name, 'sqlite:///'+db_file_name)
print('df.shape = ', df.shape)

X = df['text'] 
categories_list = [c for c in df.columns if c not in ['genre', 'text']]
categories_list.sort()
y =  df[categories_list]

df.shape =  (26161, 38)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


### 3. Prepare data
Split data into train and validation sets

In [5]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y)

### 4. Train pipeline
Train a pipeline with default parameters

In [6]:
model = Pipeline([
('vect', CountVectorizer(tokenizer=tokenize)),
('tfidf', TfidfTransformer()),
('multi_clf', MultiOutputClassifier(RandomForestClassifier()))
])

model.fit(X_train, y_train)
y_pred = model.predict(X_validation)

### 5. Evaluate your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def evaluate_results(ground_truth, predictions):

    all_scores = []
    for i, col in enumerate(categories_list):
        cls_rep = classification_report(ground_truth[col], predictions[:,i])

        # installed version 0.19 of classification_report does not allow easy dict export
        # TODO: Update the requirement on the Udacity server to sklearn 0.20+
        avg_scores = (cls_rep.split('\n')[-2]).split()[-4:]
        cur_scores = {'col':col}
        for i, f in enumerate(['precision', 'recall','f1-score','support']):
            cur_scores[f] = float(avg_scores[i]) 
        
        all_scores.append(cur_scores)

    all_scores_df = pd.DataFrame(all_scores).set_index('col')
    return all_scores_df

In [8]:
# Evaluate the first model results and give a general score based on f1-scores of all categories
all_scores_df = evaluate_results(y_validation, y_pred)
print('all_scores_df.mean = ', all_scores_df['f1-score'].mean())
print(all_scores_df.shape)
all_scores_df.head()

all_scores_df.mean =  0.926944444444
(36, 4)


,f1-score,precision,recall,support
col,,,,
aid_centers,0.98,0.98,0.99,6541.0
aid_related,0.73,0.74,0.74,6541.0
buildings,0.94,0.95,0.96,6541.0
child_alone,1.00,1.00,1.00,6541.0
clothing,0.98,0.99,0.99,6541.0


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# Create a custom functon to score models by the gridSearch 

def custom_score_func(ground_truth, predictions):
    all_scores_df = evaluate_results(ground_truth, predictions)
    mean_f1_scores = all_scores_df['f1-score'].mean()
    return mean_f1_scores

# Get the customer scorer result for the first model
custom_scorer = make_scorer(custom_score_func, greater_is_better=True)

In [10]:
# Improve the model by grid search parameters of the vect and tfidf transfomers

parameters = {
        'vect__max_df': (0.5, 1.0),
        'vect__max_features': (None, 10000),
        'tfidf__use_idf': (True, False)
        }                                                        

pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('multi_clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

cv = GridSearchCV(estimator=pipeline, param_grid=parameters,
                 verbose=10, n_jobs=8, cv=2, scoring=custom_scorer)


cv.fit(X_train, y_train)


Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None ..
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None ..
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=10000 .
[CV] tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=10000 .
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None ..
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None ..
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=10000 .
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=10000 .
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=10000, score=0.9266666666666666, total=  23.4s
[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=10000, score=0.9252777777777776, total=  23.4s
[CV] tfidf__use_idf=False, vect__max_df=0.5, vect__max_features=None .
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=10000, score=0.

[Parallel(n_jobs=8)]: Done   3 out of  16 | elapsed:   30.7s remaining:  2.2min


[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=10000, score=0.928611111111111, total=  23.6s
[CV] tfidf__use_idf=False, vect__max_df=0.5, vect__max_features=10000 
[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, score=0.9244444444444445, total=  25.7s
[CV]  tfidf__use_idf=True, vect__max_df=0.5, vect__max_features=None, score=0.9244444444444443, total=  25.7s
[CV] tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None .
[CV] tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None .


[Parallel(n_jobs=8)]: Done   5 out of  16 | elapsed:   33.4s remaining:  1.2min


[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=None, score=0.9272222222222223, total=  25.9s
[CV] tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=10000 


...

[CV]  tfidf__use_idf=False, vect__max_df=0.5, vect__max_features=None, score=0.924722222222222, total=  25.5s
[CV]  tfidf__use_idf=False, vect__max_df=0.5, vect__max_features=None, score=0.9275, total=  25.6s


[Parallel(n_jobs=8)]: Done  13 out of  16 | elapsed:  1.1min remaining:   14.6s


[CV]  tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, score=0.9244444444444445, total=  25.2s
[CV]  tfidf__use_idf=False, vect__max_df=1.0, vect__max_features=None, score=0.9263888888888889, total=  25.3s


[Parallel(n_jobs=8)]: Done  16 out of  16 | elapsed:  1.1min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'vect__max_df': (0.5, 1.0), 'vect__max_features': (None, 10000), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(custom_score_func), verbose=10)

In [11]:
# Get a unique general score of the best model found and a table of the grid search jobs

a = pd.DataFrame(cv.cv_results_)
a.sort_values(by='rank_test_score').head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tfidf__use_idf,param_vect__max_df,param_vect__max_features,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
1,16.276728,0.037835,7.128543,0.016389,True,0.5,10000,"{'tfidf__use_idf': True, 'vect__max_df': 0.5, ...",0.930556,0.926667,0.928611,0.001944,1,0.992778,0.993333,0.993056,0.000278
5,15.507567,0.049440,7.067375,0.058202,False,0.5,10000,"{'tfidf__use_idf': False, 'vect__max_df': 0.5,...",0.929167,0.926111,0.927639,0.001528,2,0.992778,0.993056,0.992917,0.000139
7,15.648455,0.116127,6.871203,0.008938,False,1,10000,"{'tfidf__use_idf': False, 'vect__max_df': 1.0,...",0.928611,0.926667,0.927639,0.000972,2,0.993056,0.992778,0.992917,0.000139


In [12]:
# params of the best model found by this grid search
cv.best_params_

{'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__max_features': 10000}

In [57]:
# score of the best model found, on the data from the training dataset 

cv.best_score_

0.92861111111111105

In [58]:
cv.best_params_

{'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__max_features': 10000}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# Results of the best model found by grid, applied to the validation dataset 

y_pred = cv.predict(X_validation)
all_scores_df = evaluate_results(y_validation, y_pred)
mean_f1_scores = all_scores_df['f1-score'].mean()
print('mean_f1_scores = ', mean_f1_scores)
print(all_scores_df.shape)
all_scores_df.head()
    

mean_f1_scores =  0.932222222222
(36, 4)


,f1-score,precision,recall,support
col,,,,
aid_centers,0.98,0.98,0.99,6541.0
aid_related,0.72,0.73,0.73,6541.0
buildings,0.94,0.95,0.96,6541.0
child_alone,1.00,1.00,1.00,6541.0
clothing,0.98,0.98,0.99,6541.0


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVC

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.neural_network import MLPRegressor

parameters2 = {
        'multi_clf__estimator': ([MultinomialNB(), KNeighborsClassifier()])
            }                                                        


pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
cv2 = GridSearchCV(estimator=pipeline2, 
                   param_grid=parameters2,
                   verbose=10, n_jobs=4, cv=2, 
                   scoring=custom_scorer)


cv2.fit(X_train, y_train)


Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] multi_clf__estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) 
[CV] multi_clf__estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) 
[CV] multi_clf__estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform') 
[CV] multi_clf__estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform') 
[CV]  multi_clf__estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), score=0.9063888888888888, total=   5.1s
[CV]  multi_clf__estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), score=0.9094444444444443, total=   5.1s


[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    7.9s
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    7.9s remaining:    7.9s


[CV]  multi_clf__estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'), score=0.9125000000000001, total= 2.0min
[CV]  multi_clf__estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'), score=0.9205555555555556, total= 2.1min


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  4.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  4.1min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'multi_clf__estimator': [MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(custom_score_func), verbose=10)

In [55]:
cv2.score(X_train, y_train)




0.9408333333333333

### 9. Export your model as a pickle file

In [54]:
import pickle
model_file_name = '../models/cv2.model.save'
pickle.dump(cv2, open(model_file_name, 'wb'))


In [56]:
loaded_model = pickle.load(open(model_file_name, 'rb'))


NameError: name 'model_file_name' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.